In [1]:
import pandas as pd
import geopandas as gpd
from timeit import repeat as timeit_repeat
from numpy import arange as np_arange

C:\Users\mohan\AppData\Local\Temp\ipykernel_22320\549825037.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
geohash_precision = 6

In [5]:
TRIPS_PATH = 'https://raw.githubusercontent.com/IsamAljawarneh/datasets/1c2a6af7dea7aa93105ac1d1d0118d07bd681d8a/data/NYC_Pilot2_PM_Part1.csv'
POLYGON_PATH = 'https://raw.githubusercontent.com/IsamAljawarneh/datasets/1c2a6af7dea7aa93105ac1d1d0118d07bd681d8a/data/nyc_polygon.geojson'

In [6]:
trips = pd.read_csv(TRIPS_PATH)
trips = trips[
    (trips['latitude']  != 0) &
    (trips['longitude'] != 0)
]

In [7]:
neighborhoods_original = gpd.read_file(POLYGON_PATH)
def group_by_sjoin(df):
    sjoined_trips_original = gpd.sjoin(df, neighborhoods_original, predicate="within")
    return sjoined_trips_original

In [11]:
for sampling_fraction in np_arange(0.1, 1.1, 0.1):
    gdf_trips = gpd.GeoDataFrame(trips, geometry=gpd.points_from_xy(trips.longitude, trips.latitude))
    gdf_trips.set_crs('epsg:4326', inplace=True)
    sampled_trips = gdf_trips.sample(frac=sampling_fraction)
    print("Best of 5 runs for sampling fraction %.1f:" % sampling_fraction, min(timeit_repeat(
        "group_by_sjoin(sampled_trips)",
        setup="from __main__ import group_by_sjoin, sampled_trips",
        repeat=5, number=1
    )))

Best of 5 runs for sampling fraction 0.1: 0.031767199994646944
Best of 5 runs for sampling fraction 0.2: 0.06452639999770327
Best of 5 runs for sampling fraction 0.3: 0.09767390000342857
Best of 5 runs for sampling fraction 0.4: 0.12907709999853978
Best of 5 runs for sampling fraction 0.5: 0.15965870000218274
Best of 5 runs for sampling fraction 0.6: 0.1961029999947641
Best of 5 runs for sampling fraction 0.7: 0.22287909999431577
Best of 5 runs for sampling fraction 0.8: 0.2599684000015259
Best of 5 runs for sampling fraction 0.9: 0.28554369999619666
Best of 5 runs for sampling fraction 1.0: 0.30459890000201995
